In [1]:
!onnxsim model.onnx model_simplified.onnx

Your model contains "Tile" ops or/and "ConstantOfShape" ops. Folding these ops 
can make the simplified model much larger. If it is not expected, please specify
"--no-large-tensor" (which will lose some optimization chances)
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                 ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add             │ 96             │ 96               │
│ ArgMax          │ 1              │ 1                │
│ Cast            │ 87             │ 30               │
│ Concat          │ 60             │ 56               │
│ Constant        │ 727            │ 335              │
│ ConstantOfShape │ 29             │ 1                │
│ Conv            │ 54             │ 54               │
│ Div             │ 55             │ 55               │
│ Einsum          │ 8              │ 8                │
│ Elu             │ 4              │ 4                │


Onnxruntimes all optmizer

In [2]:
import onnxruntime as ort
import torch
sess_options = ort.SessionOptions()

# Set graph optimization level
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

# To enable model serialization after graph optimization set this
sess_options.optimized_model_filepath = "model_simplified_optimized_optimized.onnx"

session = ort.InferenceSession("model_simplified_optimized.onnx", sess_options)

NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from model_simplified_optimized.onnx failed:Load model model_simplified_optimized.onnx failed. File doesn't exist

Quantized

In [ ]:
import onnx
from onnxruntime.quantization import quantize, quantize_dynamic, QuantType, QuantFormat, quantize_static, QuantizationMode

quantized_model = quantize_dynamic("model_simplified_optimized.onnx","model_QQ.onnx",weight_type=QuantType.QUInt8)

In [ ]:
import onnxruntime as ort
import torch
import time
def test_time(path,inputs):

    providers = [("CPUExecutionProvider")]
    sess_options = ort.SessionOptions()
    sess = ort.InferenceSession(path, sess_options=sess_options, providers=providers)
    t0 = time.time()
    for x in range(3):
      outputs2 = sess.run(None,inputs )
    t1 = time.time()
    print("total time 1 inference on CPU: ",path,(t1-t0)/3)

    return outputs2



In [ ]:

dummy_image = torch.randn(1, 1, 800,720, device="cpu").numpy()
inputs= {'image0': dummy_image,'image1': dummy_image}

o2= test_time("model.onnx",inputs)
o1= test_time("model_simplified_optimized.onnx",inputs)
o3= test_time("model_optimized.onnx",inputs)
o4= test_time("model_quantized.onnx",inputs)